In [ ]:
import sys
import numpy as np
import scipy as sc
import scipy.linalg as la
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = (8,10)
from matplotlib import cm
# from mpl_toolkits.mplot3d import Axes3D
# import plotly
# import plotly.graph_objects as go
# from scipy.optimize import minimize_scalar, minimize, shgo, basinhopping, differential_evolution, dual_annealing
#from scipy.misc import derivative
# import ipywidgets as widgets
# from ipywidgets import interact, interact_manual
# from ipywidgets import FloatProgress
from IPython.display import display, HTML, clear_output
from IPython.display import Image, SVG
#import time
# %matplotlib notebook
%load_ext tikzmagic
#https://github.com/mkrphys/ipython-tikzmagic
#
#https://github.com/mkrphys/ipython-tikzmagic/issues/18
#On Ubuntu 18.04, I ran into the same issue. It seems the problem was because ImageMagick tightened 
#their security policy so that it prevented the generation of PNG file from the PDF file generated by latex. 
#To fix this problem, I edited /etc/ImageMagick-6/policy.xml, changing the following
#
#<policy domain="coder" rights="none" pattern="PDF />
#
#to
#
#<policy domain="coder" rights="read|write" pattern="PDF />
#
#http://blog.invibe.net/posts/2015-01-20-using-tikzmagic.html
#https://stackoverflow.com/questions/45671481/how-do-i-center-the-outputs-on-a-python-jupyter-notebook
display(HTML("""
<style>
.output {
    display: flex;
    align-items: center;
    text-align: center;
}
</style>
"""))
display(HTML("<style>.container { width:80% !important; }</style>"))
#print(sc.__version__)
#print(numpy.__version__)

# Green Function Approach

## 1D Chain
We consider different topologies

In [ ]:
%%tikz -f png -s 800,500 -sc 1.0
%\tikzset{>=stealth,font=\Large};

%\fill [opacity=0.75,blue!80] (+1,+1) -- (+3,+1) -- (+3,+3) -- (+1,+3) -- cycle;

% \draw [style=help lines, step=2] (-1,-1) grid (+5,+5);
 
\filldraw [color=gray]	(0,0) circle (.1);
\filldraw [color=gray]	(2,0) circle (.1);
\filldraw [color=gray]	(4,0) circle (.1);
\filldraw [color=gray]	(6,0) circle (.1);

%\draw ( 2, 1.9) node[anchor=north] {$(i,j)_{0}$};

## Model
$$\newcommand{\OP}[1]{\hat{#1}}$$
$$\newcommand{\vectr}[1]{\boldsymbol{#1}}$$

The Hubbard model Hamiltonian for a n-site unit cell with site index $\alpha$ reads:

\begin{equation}
\OP{H} = \underbrace{\sum_{i,j}\sum_{\alpha,\beta}\sum_{\sigma} t^{\phantom{\dagger}}_{(i,\alpha),(j,\beta)} \hat{c}^{\dagger}_{i,\alpha,\sigma} \hat{c}^{\phantom{\dagger}}_{j,\beta,\sigma}}_{=: \OP{H}_0}
 + \underbrace{U \sum_{i,\alpha}\hat{n}^{\phantom{\dagger}}_{i,\alpha,\uparrow}\hat{n}^{\phantom{\dagger}}_{i,\alpha,\downarrow}}_{=: \OP{H}_\mathrm{I}}
\end{equation}

with $\hat{n}^{\phantom{\dagger}}_{i,\alpha,\sigma} =  \hat{c}^{\dagger}_{i,\alpha,\sigma} \hat{c}^{\phantom{\dagger}}_{i,\alpha,\sigma}$. The integer index set $\{i,j\}$ labels the respective cells originating from shifting the unit cell (UC) by integral multiples of the translation vectors and $\{\alpha,\alpha'\}$ the respective site index within the cell. The number of cells will be labeled $N_{\mathrm{UC}}$ and the number of basis sites $N_{\mathrm{B}}$.

### Hopping


The Fourier transformation of the general hoping term $\OP{H}_0$ reads:

\begin{equation}
 \OP{H}_0 = \sum_{\vectr{k}}\sum_{\alpha,\beta}\sum_{\sigma} \hat{c}^{\dagger}_{\vectr{k},\alpha,\sigma} \hat{c}^{\phantom{\dagger}}_{\vectr{k},\beta,\sigma} \underbrace{\left( \sum_{\vectr{\eta}_j} t^{\phantom{\dagger}}_{\alpha,\beta} e^{i\vectr{k}\cdot\vectr{\eta}_j} \right)}_{=: t_{\alpha,\beta}(\vectr{k})}.
\end{equation}

with

\begin{equation}
t_{\alpha,\beta}(\vectr{k}) = \sum_{\vectr{\eta}_j} t^{\alpha,\beta}_{\vectr{\eta}_j} e^{i\vectr{k}\cdot\vectr{\eta}_j}
\end{equation}

In [ ]:
e0 = 0.0 # local site energy
eta = 1e-4 # positive infinitesimal
wmin = -2.1; wmax = 2.1; # energy range
Nw = 1000 # number of energy points
w = np.linspace(wmin, wmax, Nw)

t = 1.0

### Undressed Propagator

\begin{equation}
    g(\omega) = \frac{1}{\omega - \epsilon_0 + i\eta} = g^r_1 (\omega) = g^r_2 (\omega)
\end{equation}

In [ ]:
g = 1.0/(w-e0+eta*1j)

### Dressed Propagator for semi-infinite Chain

\begin{equation}
    G_{11} (\omega) = \frac{1}{2g(\omega)|t|^2} \left( 1 - \sqrt{1-4|t|^2g^2(\omega)} \right)
\end{equation}

In [ ]:
Gsemi = (1.0/(2*g*2*t**2)) * (1-np.sqrt(1.0-4.0*t**2*g**2))

In [ ]:
Ginf = Gsemi/(1.0-Gsemi**2 * t**2)

In [ ]:
# Create Figure and Subplots
fig, ax = plt.subplots(1,1, figsize=(8,4), dpi=120)

# Plot
ax.plot(w, (-1.0/np.pi)*np.imag(Ginf), 'b-')
ax.plot(w, (-1.0/np.pi)*np.imag(Gsemi), 'r-')

#ax.plot([0,1,2], [0,1,4], 'b-')  # greendots

# Title, X and Y labels, X and Y Lim
ax.set_ylabel('DOS')
ax.set_xlabel('E')
ax.set_ylim(0,1.5)
ax.set_xticks([-2.0,0.0,2.0])
#ax1.set_xticklabels(['$\Gamma$','$X$'])

plt.show()

## Model
$$\newcommand{\OP}[1]{\hat{#1}}$$
$$\newcommand{\vectr}[1]{\boldsymbol{#1}}$$

The Hubbard model Hamiltonian for a single site unit cell reads:

\begin{equation}
\OP{H} = \sum_{i,j}\sum_{\sigma} t^{\phantom{\dagger}}_{i,j} \hat{c}^{\dagger}_{i,\sigma} \hat{c}^{\phantom{\dagger}}_{j,\sigma}
 + U \sum_{i}\hat{n}^{\phantom{\dagger}}_{i,\uparrow}\hat{n}^{\phantom{\dagger}}_{i,\downarrow}
\end{equation}

with $\hat{n}^{\phantom{\dagger}}_{i,\sigma} =  \hat{c}^{\dagger}_{i,\sigma} \hat{c}^{\phantom{\dagger}}_{i,\sigma}$. The integer index set $\{i,j\}$ labels the respective cells originating from shifting the unit cell (UC) by integral multiples of the translation vectors. The number of cells will be labeled $N_{\mathrm{UC}}$.

### Unitary Transformation to Local Spin Axis

\begin{equation}
\hat{c}^{\phantom{\dagger}}_{i,\sigma} = \sum_{\xi} \left[U(\theta_i,\varphi_i)\right]_{\sigma,\xi}\, \hat{d}^{\phantom{\dagger}}_{i,\xi}
\end{equation}

with $\xi\in\{+,-\}$, meaning the spin $S^{z}$ quantum number along the local spin quantization axis and:

\begin{equation}
U(\theta_i,\varphi_i) = \exp{\left(-i\frac{\varphi_{i}}{2}\sigma_{z}\right)}\exp{\left(-i\frac{\theta_{i}}{2}\sigma_{y}\right)} = 
\begin{pmatrix}
e^{-i\frac{\varphi_{i}}{2}} & 0\\
0 & e^{i\frac{\varphi_{i}}{2}}
\end{pmatrix}
\begin{pmatrix}
\cos{\left(\frac{\theta_{i}}{2}\right)} & -\sin{\left(\frac{\theta_{i}}{2}\right)}\\
\sin{\left(\frac{\theta_{i}}{2}\right)} & \cos{\left(\frac{\theta_{i}}{2}\right)}
\end{pmatrix}
=
\begin{pmatrix}
\cos{\left(\frac{\theta_{i}}{2}\right)} e^{-i\frac{\varphi_{i}}{2}} & -\sin{\left(\frac{\theta_{i}}{2}\right)} e^{-i\frac{\varphi_{i}}{2}}\\
\sin{\left(\frac{\theta_{i}}{2}\right)} e^{i\frac{\varphi_{i}}{2}} & \cos{\left(\frac{\theta_{i}}{2}\right)} e^{i\frac{\varphi_{i}}{2}}
\end{pmatrix}
\end{equation}

The transformed Hamiltonian reads

\begin{equation}
\OP{H} = \underbrace{\sum_{i,j}\sum_{\xi,\xi'} t^{\phantom{\dagger}}_{i,j} \hat{d}^{\dagger}_{i,\xi} \left[U^{\dagger}(\theta_i,\varphi_i) U(\theta_j,\varphi_j) \right]_{\xi,\xi'} \hat{d}^{\phantom{\dagger}}_{j,\xi'}}_{=: \OP{H}_0}
 + \underbrace{U \sum_{i} \hat{d}^{\dagger}_{i,+} \hat{d}^{\phantom{\dagger}}_{i,+} \hat{d}^{\dagger}_{i,-} \hat{d}^{\phantom{\dagger}}_{i,-}}_{=: \OP{H}_\mathrm{I}}
\end{equation}

In the following we consider:

\begin{equation}
t^{\phantom{\dagger}}_{i,j} = t
\end{equation}

### Interaction

The transformation of the spin base acts only on the hopping. The onsite Hubbard interaction is invariant under the transform. Therefore the derivation of the interaction part of the trail Hamiltonian as well as the mathematical structure of effective and full interaction remain the same.

\begin{equation}
\OP{H}_\mathrm{I} = U \sum_{i} \hat{d}^{\dagger}_{i,+} \hat{d}^{\phantom{\dagger}}_{i,+} \hat{d}^{\dagger}_{i,-} \hat{d}^{\phantom{\dagger}}_{i,-} = U \sum_{i} \hat{d}^{\dagger}_{i,+} \hat{d}^{\dagger}_{i,-} \hat{d}^{\phantom{\dagger}}_{i,-} \hat{d}^{\phantom{\dagger}}_{i,+} \approx \OP{H}_\mathrm{I}^{HF} = U \sum_{i} \left( \frac{1}{2} \varrho^{\phantom{\dagger}}_{i} \hat{n}^{\phantom{\dagger}}_{i} - 2\,\mathbf{m}^{\phantom{\dagger}}_{i}\cdot \OP{\mathbf{S}}_{i}^{\phantom{\dagger}} \right)
\end{equation}

with

\begin{equation}
\hat{\mathbf{S}}_{i}
= \frac{1}{2}
\begin{pmatrix}
\phantom{i}\hat{d}^{\dagger}_{i,+}\hat{d}^{\phantom{\dagger}}_{i,-}
+ \phantom{i}\hat{d}^{\dagger}_{i,-}\hat{d}^{\phantom{\dagger}}_{i,+}\\
%
i\hat{d}^{\dagger}_{i,-}\hat{d}^{\phantom{\dagger}}_{i,+}
- i\hat{d}^{\dagger}_{i,+}\hat{d}^{\phantom{\dagger}}_{i,-}\\
%
\phantom{i}\hat{d}^{\dagger}_{i,+}\hat{d}^{\phantom{\dagger}}_{i,+}
- \phantom{i}\hat{d}^{\dagger}_{i,-}\hat{d}^{\phantom{\dagger}}_{i,-}\\
\end{pmatrix}
\end{equation}

and 

\begin{equation}
\hat{n}^{\phantom{\dagger}}_{i} = \hat{d}^{\dagger}_{i,+} \hat{d}^{\phantom{\dagger}}_{i,+} + \hat{d}^{\dagger}_{i,-} \hat{d}^{\phantom{\dagger}}_{i,-}
\end{equation}

Restriction to $S^{z}$ - component and 1 site unit cell:

\begin{align}
\varrho_{i} &\rightarrow \varrho\\
m^{z}_{i} &\rightarrow m^{z}
\end{align}

yielding:

\begin{equation}
\OP{H}_\mathrm{I}^{HF} = U \sum_{i} \left( \frac{1}{2} \varrho^{\phantom{\dagger}} \left(\hat{d}^{\dagger}_{i,+} \hat{d}^{\phantom{\dagger}}_{i,+} + \hat{d}^{\dagger}_{i,-} \hat{d}^{\phantom{\dagger}}_{i,-}\right) - m^{z}\left(\hat{d}^{\dagger}_{i,+}\hat{d}^{\phantom{\dagger}}_{i,+}
-\hat{d}^{\dagger}_{i,-}\hat{d}^{\phantom{\dagger}}_{i,-} \right) \right)
\end{equation}

Fourier Transformation:

\begin{equation}
\OP{H}_\mathrm{I}^{HF} = U \sum_{\vectr{k}} \left( \frac{1}{2} \varrho^{\phantom{\dagger}} \left(\hat{d}^{\dagger}_{\vectr{k},+} \hat{d}^{\phantom{\dagger}}_{\vectr{k},+} + \hat{d}^{\dagger}_{\vectr{k},-} \hat{d}^{\phantom{\dagger}}_{\vectr{k},-}\right) - m^{z}\left(\hat{d}^{\dagger}_{\vectr{k},+}\hat{d}^{\phantom{\dagger}}_{\vectr{k},+}
-\hat{d}^{\dagger}_{\vectr{k},-}\hat{d}^{\phantom{\dagger}}_{\vectr{k},-} \right) \right)
\end{equation}